## एम्बेडिंग्ज

आपल्या मागील उदाहरणात, आम्ही `vocab_size` लांबीच्या उच्च-आयामी बॅग-ऑफ-वर्ड्स व्हेक्टरवर कार्य केले, आणि आम्ही कमी-आयामी स्थानिक प्रतिनिधित्व व्हेक्टरमधून विरळ वन-हॉट प्रतिनिधित्वामध्ये स्पष्टपणे रूपांतरित करत होतो. हे वन-हॉट प्रतिनिधित्व मेमरीसाठी कार्यक्षम नाही, याशिवाय प्रत्येक शब्द स्वतंत्रपणे हाताळला जातो, म्हणजेच वन-हॉट एन्कोड केलेले व्हेक्टर शब्दांमधील कोणतेही अर्थपूर्ण साम्य व्यक्त करत नाहीत.

या युनिटमध्ये, आपण **News AG** डेटासेटचा अभ्यास सुरू ठेवू. सुरुवात करण्यासाठी, डेटा लोड करूया आणि मागील नोटबुकमधील काही संकल्पना घेऊया.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## एम्बेडिंग म्हणजे काय?

**एम्बेडिंग** ही कल्पना म्हणजे शब्दांचे प्रतिनिधित्व कमी-आयामी घन व्हेक्टरद्वारे करणे, जे काही प्रमाणात शब्दाचा अर्थ प्रतिबिंबित करतात. आपण नंतर अर्थपूर्ण शब्द एम्बेडिंग कसे तयार करायचे यावर चर्चा करू, पण सध्या एम्बेडिंगला शब्द व्हेक्टरची आयाम कमी करण्याचा एक मार्ग म्हणून विचार करूया.

तर, एम्बेडिंग लेयर एक शब्द इनपुट म्हणून घेईल आणि निर्दिष्ट `embedding_size` चा आउटपुट व्हेक्टर तयार करेल. एका अर्थाने, हे `Linear` लेयरसारखेच आहे, पण एक-हॉट एन्कोडेड व्हेक्टर घेण्याऐवजी, ते शब्द क्रमांक इनपुट म्हणून घेऊ शकेल.

आपल्या नेटवर्कमध्ये एम्बेडिंग लेयर प्रथम लेयर म्हणून वापरल्याने, आपण बॅग-ऑफ-वर्ड्स मॉडेलवरून **एम्बेडिंग बॅग** मॉडेलकडे स्विच करू शकतो, जिथे आपण प्रथम आपल्या मजकुरातील प्रत्येक शब्द संबंधित एम्बेडिंगमध्ये रूपांतरित करतो आणि नंतर त्या सर्व एम्बेडिंगवर काही एकत्रित फंक्शन गणना करतो, जसे की `sum`, `average` किंवा `max`.

![पाच अनुक्रम शब्दांसाठी एम्बेडिंग वर्गीकरणकर्ता दर्शवणारी प्रतिमा.](../../../../../translated_images/mr/embedding-classifier-example.b77f021a7ee67eee.webp)

आपले वर्गीकरणकर्ता न्यूरल नेटवर्क एम्बेडिंग लेयरने सुरू होईल, त्यानंतर एकत्रीकरण लेयर आणि त्यावर लीनियर वर्गीकरणकर्ता असेल:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### बदलत्या व्हेरिएबल अनुक्रम आकाराशी सामना करणे

या आर्किटेक्चरमुळे, आमच्या नेटवर्कसाठी मिनीबॅचेस विशिष्ट पद्धतीने तयार करणे आवश्यक होईल. मागील युनिटमध्ये, बॅग-ऑफ-वर्ड्स वापरताना, मिनीबॅचमधील सर्व BoW टेन्सर्सचा आकार `vocab_size` इतकाच होता, आपल्या मजकूर अनुक्रमाच्या वास्तविक लांबीची पर्वा न करता. जेव्हा आपण वर्ड एम्बेडिंगकडे जातो, तेव्हा प्रत्येक मजकूर नमुन्यात वेगवेगळ्या संख्येने शब्द असतील, आणि ते नमुने मिनीबॅचमध्ये एकत्र करताना आपल्याला काही पॅडिंग लागू करावे लागेल.

हे `collate_fn` फंक्शन डेटा स्रोताला प्रदान करून त्याच तंत्राचा वापर करून करता येते:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### एम्बेडिंग वर्गीकरण प्रशिक्षण

आता आपण योग्य डेटालोडर परिभाषित केला आहे, आपण मागील युनिटमध्ये परिभाषित केलेल्या प्रशिक्षण फंक्शनचा वापर करून मॉडेल प्रशिक्षण देऊ शकतो:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **टीप**: वेळेच्या बचतीसाठी आम्ही येथे फक्त २५,००० नोंदींसाठी प्रशिक्षण देत आहोत (एक पूर्ण युगापेक्षा कमी), परंतु तुम्ही प्रशिक्षण सुरू ठेवू शकता, अनेक युगांसाठी प्रशिक्षण देण्यासाठी एक फंक्शन लिहू शकता आणि उच्च अचूकता मिळवण्यासाठी शिक्षण दर पॅरामीटरसह प्रयोग करू शकता. तुम्ही सुमारे ९०% अचूकतेपर्यंत पोहोचू शकता.


### EmbeddingBag लेयर आणि बदलत्या लांबीच्या अनुक्रमाचे प्रतिनिधित्व

मागील आर्किटेक्चरमध्ये, सर्व अनुक्रम समान लांबीचे करण्यासाठी त्यांना पॅड करणे आवश्यक होते, जेणेकरून ते मिनीबॅचमध्ये बसतील. बदलत्या लांबीच्या अनुक्रमांचे प्रतिनिधित्व करण्याचा हा सर्वात कार्यक्षम मार्ग नाही - दुसरा दृष्टिकोन म्हणजे **ऑफसेट** व्हेक्टर वापरणे, जो एका मोठ्या व्हेक्टरमध्ये संग्रहित केलेल्या सर्व अनुक्रमांचे ऑफसेट ठेवेल.

![ऑफसेट अनुक्रमाचे प्रतिनिधित्व दर्शवणारी प्रतिमा](../../../../../translated_images/mr/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: वरील चित्रात, आम्ही अक्षरांच्या अनुक्रमाचे प्रदर्शन केले आहे, परंतु आमच्या उदाहरणात आम्ही शब्दांच्या अनुक्रमांवर काम करत आहोत. तथापि, ऑफसेट व्हेक्टरसह अनुक्रमांचे प्रतिनिधित्व करण्याचा सामान्य तत्त्व समान राहतो.

ऑफसेट प्रतिनिधित्वासह काम करण्यासाठी, आम्ही [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) लेयर वापरतो. हे `Embedding` सारखेच आहे, परंतु ते कंटेंट व्हेक्टर आणि ऑफसेट व्हेक्टर इनपुट म्हणून घेतो, आणि त्यात सरासरीकरण लेयर देखील समाविष्ट आहे, जे `mean`, `sum` किंवा `max` असू शकते.

येथे `EmbeddingBag` वापरणारे सुधारित नेटवर्क आहे:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

डेटासेट प्रशिक्षणासाठी तयार करण्यासाठी, आपल्याला ऑफसेट व्हेक्टर तयार करणारी रूपांतरण फंक्शन प्रदान करावी लागेल:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

आता लक्षात घ्या, की सर्व पूर्वीच्या उदाहरणांपेक्षा वेगळे, आमचे नेटवर्क आता दोन पॅरामीटर्स स्वीकारते: डेटा व्हेक्टर आणि ऑफसेट व्हेक्टर, जे वेगवेगळ्या आकाराचे आहेत. त्याचप्रमाणे, आमचा डेटा लोडर देखील आम्हाला 2 ऐवजी 3 मूल्ये प्रदान करतो: मजकूर आणि ऑफसेट व्हेक्टर दोन्ही वैशिष्ट्ये म्हणून प्रदान केले जातात. त्यामुळे, आम्हाला आमच्या प्रशिक्षण फंक्शनमध्ये थोडासा बदल करणे आवश्यक आहे:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## सिमॅंटिक एम्बेडिंग्स: वर्ड2व्हेक

आपल्या मागील उदाहरणात, मॉडेलच्या एम्बेडिंग लेयरने शब्दांना व्हेक्टर स्वरूपात मॅप करणे शिकले, परंतु या स्वरूपात फारसा सिमॅंटिक अर्थ नव्हता. असे व्हेक्टर स्वरूप शिकणे चांगले ठरेल, ज्यामध्ये समान शब्द किंवा समानार्थी शब्द अशा व्हेक्टरना सादर करतील जे काही व्हेक्टर अंतर (उदा. युक्लिडियन अंतर) यांच्या दृष्टीने एकमेकांच्या जवळ असतील.

हे साध्य करण्यासाठी, आपल्याला मोठ्या प्रमाणातील मजकुरावर विशिष्ट पद्धतीने आपले एम्बेडिंग मॉडेल प्री-ट्रेन करावे लागेल. सिमॅंटिक एम्बेडिंग्स ट्रेन करण्याच्या पहिल्या पद्धतींपैकी एक म्हणजे [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). हे दोन मुख्य आर्किटेक्चर्सवर आधारित आहे, जे शब्दांचे वितरित प्रतिनिधित्व तयार करण्यासाठी वापरले जातात:

 - **कंटिन्युअस बॅग-ऑफ-वर्ड्स** (CBoW) — या आर्किटेक्चरमध्ये, आपण मॉडेलला आजूबाजूच्या संदर्भातून एखादा शब्द भाकीत करण्यासाठी ट्रेन करतो. दिलेल्या ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ मध्ये, मॉडेलचे उद्दिष्ट $(W_{-2},W_{-1},W_1,W_2)$ वरून $W_0$ भाकीत करणे आहे.
 - **कंटिन्युअस स्किप-ग्राम** हे CBoW च्या उलट आहे. मॉडेल सध्याचा शब्द भाकीत करण्यासाठी संदर्भातील आजूबाजूच्या शब्दांचा वापर करते.

CBoW जलद आहे, तर स्किप-ग्राम थोडा धीमा आहे, परंतु दुर्मिळ शब्दांचे प्रतिनिधित्व करण्याचे काम अधिक चांगल्या प्रकारे करतो.

![CBoW आणि स्किप-ग्राम अल्गोरिदम्स शब्दांना व्हेक्टरमध्ये रूपांतरित करण्यासाठी दाखवणारी प्रतिमा.](../../../../../translated_images/mr/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News डेटासेटवर प्री-ट्रेन केलेल्या word2vec एम्बेडिंगसह प्रयोग करण्यासाठी, आपण **gensim** लायब्ररीचा वापर करू शकतो. खाली 'neural' या शब्दाशी सर्वाधिक समान शब्द शोधले आहेत:

> **Note:** जेव्हा तुम्ही प्रथमच शब्दांचे व्हेक्टर तयार करता, तेव्हा त्यांना डाउनलोड करण्यात काही वेळ लागू शकतो!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


आम्ही शब्दातून व्हेक्टर एम्बेडिंग्स देखील गणना करू शकतो, ज्याचा वापर वर्गीकरण मॉडेल प्रशिक्षणासाठी केला जाऊ शकतो (स्पष्टतेसाठी आम्ही फक्त व्हेक्टरचे पहिले 20 घटक दाखवतो):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

दोन्ही CBoW आणि Skip-Grams हे "भाकीत करणारे" एम्बेडिंग्स आहेत, कारण ते फक्त स्थानिक संदर्भांवर आधारित असतात. Word2Vec जागतिक संदर्भाचा फायदा घेत नाही.

**FastText** हे Word2Vec वर आधारित आहे, ज्यामध्ये प्रत्येक शब्दासाठी आणि त्या शब्दामध्ये आढळणाऱ्या अक्षर n-grams साठी व्हेक्टर प्रतिनिधित्व शिकवले जाते. या प्रतिनिधित्वांचे मूल्य प्रत्येक प्रशिक्षण टप्प्यावर एका व्हेक्टरमध्ये सरासरी काढले जाते. जरी यामुळे प्री-ट्रेनिंगसाठी अतिरिक्त गणना लागते, तरीही हे शब्द एम्बेडिंग्सना उप-शब्द माहिती एन्कोड करण्यास सक्षम करते.

आणखी एक पद्धत, **GloVe**, सह-अस्तित्व मॅट्रिक्सच्या संकल्पनेचा उपयोग करते आणि सह-अस्तित्व मॅट्रिक्सला अधिक अभिव्यक्त आणि नॉन-लिनियर शब्द व्हेक्टरमध्ये विघटित करण्यासाठी न्यूरल पद्धतींचा वापर करते.

तुम्ही एम्बेडिंग्स बदलून FastText आणि GloVe वापरून उदाहरण खेळू शकता, कारण gensim विविध शब्द एम्बेडिंग मॉडेल्सला समर्थन देते.


## PyTorch मध्ये पूर्व-प्रशिक्षित एम्बेडिंग्ज वापरणे

वरील उदाहरण बदलून आपण आपल्या एम्बेडिंग लेयरमधील मॅट्रिक्सला Word2Vec सारख्या अर्थपूर्ण एम्बेडिंग्जने पूर्व-भरू शकतो. आपल्याला हे लक्षात घ्यावे लागेल की पूर्व-प्रशिक्षित एम्बेडिंग आणि आपल्या टेक्स्ट कॉर्पसचे शब्दसंग्रह (vocabularies) बहुधा जुळणार नाहीत, त्यामुळे गहाळ शब्दांसाठी वेट्स (weights) यादृच्छिक (random) मूल्यांनी प्रारंभ करू:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

आपल्या बाबतीत, आम्हाला अचूकतेत मोठी वाढ दिसत नाही, ज्याचे मुख्य कारण वेगवेगळ्या शब्दसंग्रहांमुळे असावे.  
वेगवेगळ्या शब्दसंग्रहांच्या समस्येवर मात करण्यासाठी, आपण खालील उपायांपैकी एक वापरू शकतो:  
* आपल्या शब्दसंग्रहावर word2vec मॉडेल पुन्हा प्रशिक्षित करा  
* पूर्व-प्रशिक्षित word2vec मॉडेलमधील शब्दसंग्रहासह आपला डेटासेट लोड करा. डेटासेट लोड करताना वापरला जाणारा शब्दसंग्रह निर्दिष्ट केला जाऊ शकतो.  

दुसरा दृष्टिकोन सोपा वाटतो, विशेषतः कारण PyTorch `torchtext` फ्रेमवर्कमध्ये एम्बेडिंगसाठी अंगभूत समर्थन आहे. उदाहरणार्थ, आपण खालीलप्रमाणे GloVe-आधारित शब्दसंग्रह तयार करू शकतो:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


लोड केलेल्या शब्दसंग्रहामध्ये खालील मूलभूत क्रिया आहेत:
* `vocab.stoi` शब्दकोश आपल्याला शब्द त्याच्या शब्दकोश निर्देशांकामध्ये रूपांतरित करण्याची परवानगी देतो
* `vocab.itos` याउलट करते - संख्या शब्दामध्ये रूपांतरित करते
* `vocab.vectors` हा एम्बेडिंग व्हेक्टरचा सरणी आहे, त्यामुळे एखाद्या शब्दाचा `s` एम्बेडिंग मिळवण्यासाठी आपल्याला `vocab.vectors[vocab.stoi[s]]` वापरावे लागेल

येथे एम्बेडिंग्समध्ये फेरफार करण्याचे उदाहरण दिले आहे, जेणेकरून समीकरण **kind-man+woman = queen** सिद्ध करता येईल (हे कार्य करण्यासाठी मला गुणांक थोडासा बदलावा लागला):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

डेटासेटला GloVe शब्दसंग्रहाचा वापर करून एन्कोड करण्यासाठी, आम्हाला प्रथम त्या एम्बेडिंग्सचा वापर करून वर्गीकरणकर्ता प्रशिक्षित करावा लागेल:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

जसे आपण वर पाहिले, सर्व व्हेक्टर एम्बेडिंग्ज `vocab.vectors` मॅट्रिक्समध्ये संग्रहित केल्या जातात. साध्या कॉपींगचा वापर करून त्या वेट्स एम्बेडिंग लेयरच्या वेट्समध्ये लोड करणे खूप सोपे होते:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

आम्ही अचूकतेमध्ये लक्षणीय वाढ पाहत नाही याचे एक कारण म्हणजे आमच्या डेटासेटमधील काही शब्द प्री-ट्रेन केलेल्या GloVe शब्दसंग्रहात अनुपस्थित आहेत, आणि त्यामुळे त्याकडे मूलत: दुर्लक्ष केले जाते. या गोष्टीवर मात करण्यासाठी, आम्ही आमच्या डेटासेटवर स्वतःचे एम्बेडिंग्स ट्रेन करू शकतो.


## संदर्भात्मक एम्बेडिंग्स

पारंपरिक प्रीट्रेन केलेल्या एम्बेडिंग्स जसे की Word2Vec यांची एक महत्त्वाची मर्यादा म्हणजे शब्द अर्थ अस्पष्टता (word sense disambiguation) ची समस्या. प्रीट्रेन केलेल्या एम्बेडिंग्स काही प्रमाणात शब्दांचा संदर्भातील अर्थ पकडू शकतात, परंतु प्रत्येक संभाव्य अर्थ एका एम्बेडिंगमध्येच समाविष्ट केला जातो. यामुळे डाउनस्ट्रीम मॉडेल्समध्ये समस्या निर्माण होऊ शकतात, कारण अनेक शब्द, जसे की 'play', वेगवेगळ्या संदर्भांमध्ये वेगवेगळे अर्थ दर्शवतात.

उदाहरणार्थ, 'play' या शब्दाचा खालील दोन वाक्यांमध्ये अर्थ पूर्णपणे वेगळा आहे:
- मी थिएटरमध्ये एक **play** पाहायला गेलो.
- जॉनला त्याच्या मित्रांसोबत **play** करायचे आहे.

वरील प्रीट्रेन केलेल्या एम्बेडिंग्स 'play' या शब्दाचे दोन्ही अर्थ एकाच एम्बेडिंगमध्ये दर्शवतात. ही मर्यादा दूर करण्यासाठी, आपल्याला **भाषा मॉडेल** आधारित एम्बेडिंग्स तयार करणे आवश्यक आहे, जे मोठ्या प्रमाणातील मजकूरावर प्रशिक्षित केले जाते आणि *जाणते* की शब्द वेगवेगळ्या संदर्भांमध्ये कसे एकत्र ठेवले जाऊ शकतात. संदर्भात्मक एम्बेडिंग्सवर चर्चा करणे या ट्यूटोरियलच्या कक्षेबाहेर आहे, परंतु आपण पुढील युनिटमध्ये भाषा मॉडेल्सबद्दल बोलताना त्याकडे परत येऊ.



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार नाही.
